In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Normalizer , scale
from sklearn.model_selection import GridSearchCV , KFold , cross_val_score
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error 
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from sklearn.utils import shuffle
from keras.utils import plot_model
%matplotlib inline

In [2]:
checkin_cols = ['user_id', 'poi_id', 'timestamp', 'timezone']
checkins = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_Checkins.csv', sep=',', names=checkin_cols, encoding='latin-1')

venue_cols = ['poi_id', 'latitude', 'longitude', 'category', 'country_code']
pois = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_POIs.csv', sep=',', names=venue_cols, encoding='latin-1')

C:\Users\lasse\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (0,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\lasse\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
country = pd.DataFrame(pois, columns=['poi_id', 'country_code'])
countrycode = checkins.merge(country, on='poi_id')

denmark = countrycode.where(countrycode['country_code'] == 'DK').dropna()

print("Number of checkins in denmark")
len(denmark)

users_denmark = denmark.copy()
users_denmark.drop_duplicates(subset="user_id", keep = 'first', inplace = True)

print("Number of unique users in denmark")
len(users_denmark)

Number of checkins in denmark
Number of unique users in denmark


981

In [9]:
print(denmark)

         user_id                    poi_id                       timestamp  \
524329    233919  4c2f45da66e40f47aa8ec18b  Tue Apr 03 18:19:44 +0000 2012   
731652    190585  4adcdb00f964a520775e21e3  Tue Apr 03 18:30:09 +0000 2012   
731653     24779  4adcdb00f964a520775e21e3  Sun Apr 29 11:33:55 +0000 2012   
731654     30835  4adcdb00f964a520775e21e3  Wed Jun 27 14:49:26 +0000 2012   
731655      3884  4adcdb00f964a520775e21e3  Mon Nov 12 11:53:23 +0000 2012   
...          ...                       ...                             ...   
33257607   45339  4ae1d14ef964a520ac8721e3  Sat Sep 14 20:42:29 +0000 2013   
33257719  241203  5234d16f11d2aa27377c1d6d  Sat Sep 14 21:14:11 +0000 2013   
33261149  201974  4c4ec7deea24c9b6e7143a16  Mon Sep 16 09:34:09 +0000 2013   
33261538   44648  4adcdafcf964a5205a5d21e3  Mon Sep 16 10:50:16 +0000 2013   
33262742  216973  4b48e2c6f964a520be5b26e3  Mon Sep 16 16:52:20 +0000 2013   

         timezone country_code  
524329        120           DK

In [10]:
denmarksave = pd.DataFrame(denmark, columns=['user_id', 'poi_id', 'timestamp', 'timezone'])
denmarksave.to_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\Den_checkins.csv', sep=',', index=['user_id', 'poi_id', 'timestamp', 'timezone'])
poi_id = pd.DataFrame(denmark, columns=['poi_id'])
pois_poi_id = pois.merge(poi_id, on='poi_id')
pois_poi_id.drop_duplicates(subset="poi_id", keep = 'first', inplace = True)
pois_poi_id.to_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\Den_pois.csv', sep=',', index=['poi_id', 'latitude', 'longitude', 'category', 'country_code'])


In [5]:
country = pd.DataFrame(pois, columns=['poi_id', 'country_code'])
countrycode = checkins.merge(country, on='poi_id')

sweden = countrycode.where(countrycode['country_code'] == 'SE').dropna()

print("Number of checkins in Sweden")
len(sweden)

users_sweden = sweden.copy()
users_sweden.drop_duplicates(subset="user_id", keep = 'first', inplace = True)

print("Number of unique users in Sweden")
len(users_sweden)

Number of checkins in Sweden
Number of unique users in Sweden


1123

In [11]:
denswe = pd.concat([denmark, sweden], axis=0)
denmarksave = pd.DataFrame(denswe, columns=['user_id', 'poi_id', 'timestamp', 'timezone'])
denmarksave.to_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\DenSwe_checkins.csv', sep=',', index=['user_id', 'poi_id', 'timestamp', 'timezone'])
poi_id = pd.DataFrame(denswe, columns=['poi_id'])
pois_poi_id = pois.merge(poi_id, on='poi_id')
pois_poi_id.drop_duplicates(subset="poi_id", keep = 'first', inplace = True)
pois_poi_id.to_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\DenSwe_pois.csv', sep=',', index=['poi_id', 'latitude', 'longitude', 'category', 'country_code'])


In [7]:
country = pd.DataFrame(pois, columns=['poi_id', 'country_code'])
countrycode = checkins.merge(country, on='poi_id')

germany = countrycode.where(countrycode['country_code'] == 'DE').dropna()

print("Number of checkins in Germany")
len(germany)

users_germany = germany.copy()
users_germany.drop_duplicates(subset="user_id", keep = 'first', inplace = True)

print("Number of unique users in Germany")
len(users_germany)

Number of checkins in Germany
Number of unique users in Germany


7901

In [12]:
densweger = pd.concat([denswe, germany], axis=0)
denmarksave = pd.DataFrame(densweger, columns=['user_id', 'poi_id', 'timestamp', 'timezone'])
denmarksave.to_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\DenSweGer_checkins.csv', sep=',', index=['user_id', 'poi_id', 'timestamp', 'timezone'])
poi_id = pd.DataFrame(densweger, columns=['poi_id'])
pois_poi_id = pois.merge(poi_id, on='poi_id')
pois_poi_id.drop_duplicates(subset="poi_id", keep = 'first', inplace = True)
pois_poi_id.to_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\DenSweGer_pois.csv', sep=',', index=['poi_id', 'latitude', 'longitude', 'category', 'country_code'])


In [9]:
densweger.to_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\DenSweGer.csv', sep=',', index=['user_id', 'poi_id', 'timestamp', 'timezone', 'country_code'])

30228

In [12]:
#denmark.join(sweden)


In [15]:
len(densweger)

208726

In [16]:
print(densweger)

         user_id                    poi_id                       timestamp  \
524329    233919  4c2f45da66e40f47aa8ec18b  Tue Apr 03 18:19:44 +0000 2012   
731652    190585  4adcdb00f964a520775e21e3  Tue Apr 03 18:30:09 +0000 2012   
731653     24779  4adcdb00f964a520775e21e3  Sun Apr 29 11:33:55 +0000 2012   
731654     30835  4adcdb00f964a520775e21e3  Wed Jun 27 14:49:26 +0000 2012   
731655      3884  4adcdb00f964a520775e21e3  Mon Nov 12 11:53:23 +0000 2012   
...          ...                       ...                             ...   
33263019  258743  4b4e1154f964a5205ce026e3  Mon Sep 16 18:22:47 +0000 2013   
33263227    9437  50c658d3e4b0dbacde326f19  Mon Sep 16 19:50:26 +0000 2013   
33263331  128208  4bed1cf250669c74965248fb  Mon Sep 16 20:39:04 +0000 2013   
33263405  136828  523776d511d2d88ae9c922cc  Mon Sep 16 21:24:26 +0000 2013   
33263628  252648  4fd2309ee4b0362dd3c65ddc  Mon Sep 16 23:22:43 +0000 2013   

         timezone country_code  
524329        120           DK

In [18]:
densweger.to_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\DenSweGer.csv', sep=',', index=['user_id', 'poi_id', 'timestamp', 'timezone', 'country_code'])

In [19]:
country = pd.DataFrame(pois, columns=['poi_id', 'country_code'])
countrycode = checkins.merge(country, on='poi_id')

netherlands = countrycode.where(countrycode['country_code'] == 'NL').dropna()

print("Number of checkins in Sweden")
len(netherlands)

users_netherlands = sweden.copy()
users_netherlands.drop_duplicates(subset="user_id", keep = 'first', inplace = True)

print("Number of unique users in Sweden")
len(users_netherlands)

Number of checkins in Sweden
Number of unique users in Sweden


1123

In [25]:
len(checkins)

33263634

In [21]:
densweger = pd.concat([densweger, netherlands], axis=0)

In [22]:
len(densweger)

423996

In [23]:
densweger.to_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\DenSweGerNet.csv', sep=',', index=['user_id', 'poi_id', 'timestamp', 'timezone', 'country_code'])

In [ ]:
country = pd.DataFrame(pois, columns=['poi_id', 'country_code'])
countrycode = checkins.merge(country, on='poi_id')

netherlands = countrycode.where(countrycode['country_code'] == 'NL').dropna()

print("Number of checkins in Sweden")
len(netherlands)

users_netherlands = sweden.copy()
users_netherlands.drop_duplicates(subset="user_id", keep = 'first', inplace = True)

print("Number of unique users in Sweden")
len(users_netherlands)

In [6]:
checkin_cols = ['user_id', 'poi_id', 'timestamp', 'timezone', 'country_code']
checkins = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\DenSweGerNet.csv', sep=',', names=checkin_cols, encoding='latin-1')

C:\Users\lasse\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (1,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


            user_id                    poi_id                       timestamp  \
NaN         user_id                    poi_id                       timestamp   
524329.0     233919  4c2f45da66e40f47aa8ec18b  Tue Apr 03 18:19:44 +0000 2012   
731652.0     190585  4adcdb00f964a520775e21e3  Tue Apr 03 18:30:09 +0000 2012   
731653.0      24779  4adcdb00f964a520775e21e3  Sun Apr 29 11:33:55 +0000 2012   
731654.0      30835  4adcdb00f964a520775e21e3  Wed Jun 27 14:49:26 +0000 2012   
...             ...                       ...                             ...   
33263060.0    91075  4c9dc0f2d3c2b60cc657c4bc  Mon Sep 16 18:38:55 +0000 2013   
33263113.0   185228  4ec4c1df77c8d69510ee11ed  Mon Sep 16 18:59:30 +0000 2013   
33263302.0   258948  4cf520ff6195721e82cf55c1  Mon Sep 16 20:26:11 +0000 2013   
33263370.0   209871  4bcf2a839854d13a977ef54d  Mon Sep 16 21:03:21 +0000 2013   
33263422.0   170216  4a27db7ff964a52024941fe3  Mon Sep 16 21:34:37 +0000 2013   

            timezone  count

In [8]:
venue_cols = ['poi_id', 'latitude', 'longitude', 'category', 'country_code']
pois = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_POIs.csv', sep=',', names=venue_cols, encoding='latin-1')

C:\Users\lasse\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [19]:
poi_id = pd.DataFrame(checkins, columns=['poi_id'])
print(poi_id)

                              poi_id
NaN                           poi_id
524329.0    4c2f45da66e40f47aa8ec18b
731652.0    4adcdb00f964a520775e21e3
731653.0    4adcdb00f964a520775e21e3
731654.0    4adcdb00f964a520775e21e3
...                              ...
33263060.0  4c9dc0f2d3c2b60cc657c4bc
33263113.0  4ec4c1df77c8d69510ee11ed
33263302.0  4cf520ff6195721e82cf55c1
33263370.0  4bcf2a839854d13a977ef54d
33263422.0  4a27db7ff964a52024941fe3

[423997 rows x 1 columns]


In [20]:
pois_poi_id = pois.merge(poi_id, on='poi_id')

In [22]:
pois_poi_id.drop_duplicates(subset="poi_id", keep = 'first', inplace = True)

In [23]:
print(pois_poi_id)

                          poi_id   latitude  longitude             category  \
0       4a120aa1f964a5206d771fe3  50.948524   6.943284          Event Space   
12      4a12fc13f964a520aa771fe3  50.947711   6.921073            Gastropub   
27      4a143153f964a52016781fe3  51.035462   7.011976     Department Store   
29      4a15ae1bf964a520b5781fe3  50.936197   6.939741           Restaurant   
36      4a172db1f964a52039791fe3  50.946437    6.94588                  Bar   
...                          ...        ...        ...                  ...   
423991  5236ddfc11d211f58f537eeb  53.555296   9.986536  Government Building   
423992  523710b211d25d0fa49c397e   51.83181   5.905503               Office   
423993  5237350f11d2133b5045fef8  52.508619  13.398192         Tech Startup   
423994  52374ad711d2a8703213e9e3  52.355326   4.904294      Assisted Living   
423995  523776d511d2d88ae9c922cc  48.681974   9.286265       Home (private)   

       country_code  
0                DE  
12     

In [30]:
save = pd.DataFrame(checkins, columns=['user_id', 'poi_id', 'timestamp', 'timezone'])
#save.to_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\DenSweGerNet_checkins.csv', sep=',', index=['user_id', 'poi_id', 'timestamp', 'timezone'])

In [29]:
#len(checkins)

423997

In [31]:
save.drop_duplicates(subset="user_id", keep = 'first', inplace = True)

In [32]:
len(save)

18160